 # AUTOMATIC CHORD-SCALE DETECTION & STUDENT PERFORMANCE ASSESMENT FOR JAZZ IMPROVISATION
 
 ## PART 1: DOWNLOAD DATASET & FEATURE EXTRACTION

In [1]:
import sys,os, pickle
sys.path.append(os.path.join(os.path.dirname('__file__'), 'freesound-python/'))

import numpy as np

import freesound
client = freesound.FreesoundClient()
client.set_token("1Dio5sf5Kq7uxPPi4WSQljr4EncEzRjuVdlMV8z8","token")

from IPython.display import Audio
from IPython.core.display import display, HTML, Image

from utilities import *


#CHORD-SCALE DATASET on freesound.org
freesound_PACK_ID = 24075

DOWNLOADSOUNDS = False ### SET 'True' FOR DOWNLOADING THE SOUNDS FROM FREESOUND, SET 'False' IF SOUNDS ALREADY EXIST

#Directory for audio files
audioDir = 'audio/'
#Directory for audio segment annotations
annotationsDir = 'scale_annotations/'
#Directory for Feature Data (.csv and pickle files)
dataDir = 'data/'

#ANALYSIS PARAMETERS FOR FEATURE EXTRACTION

params=AnalysisParams(200,100,'hann',2048,44100,12)

#INITIATE PROPER DATA STRUCTURE
dataDictionary = createDataStructure(annotationsDir) 

### DOWNLOAD THE 'CHORD-SCALE PACK (OR dataset)' FROM FREESOUND

In [2]:
if os.path.exists(audioDir) != 1:
    os.mkdir(audioDir)

pack = client.get_pack(freesound_PACK_ID)
print("Pack name:", pack.name)

results_pager = pack.get_sounds(page_size=50,fields="id,name,username,duration,previews")
 
scaleTypes = []    
TotalDuration = []
for sound in results_pager:
    
    SOUND_NAME = sound.name.split('.')[0]
    TotalDuration.append(sound.duration)
    if DOWNLOADSOUNDS:
        sound.retrieve_preview(audioDir,SOUND_NAME+".mp3")   
    
    ## dataDict[sound.name][0] has the 'duration' & 'freesoundid' information
    dataDictionary[SOUND_NAME][0]['duration']= sound.duration
    dataDictionary[SOUND_NAME][0]['freesoundid'] = sound.id

print("Number of sounds :", results_pager.count)   
print("Total duration of the dataset : " , np.sum(TotalDuration)/60.0, 'min')

Pack name: Chord-Scale Dataset


KeyboardInterrupt: 

### DISPLAY DATA

In [ ]:
scaleType = 'mixolydian'

SoundsList = []
freesoundids = []
for sound in dataDictionary.items():
    if scaleType in sound[0].split('_')[1]:
        SoundsList.append(sound)
        freesoundids.append(sound[1][0]['freesoundid'])
        
html = "<h3 style='color:#bb;'>%i) %s</h3>" % (1,scaleType)
html += "<h4>Example sound:</h4><br>"
html += generate_html_with_sound_examples(freesoundids)
html += "<br><br>"
display(HTML(html))

### FEATURE EXTRACTION

#### 1) HARMONIC PITCH CLASS PROFILES (HPCP) (Gomez, 2006)

In [4]:
'''
DEFINE PARAMETERS FOR FEATURE EXTRACTION

    - AnalysisParams:
        windowSize = 200ms
        hopSize = 100ms
        windowFunction = 'hanning'
        fftN (fftSize) = 2048
        fs (sampleRate) = 44100
        numBins (HPCP Resolution) = 12 bins/octave
        
'''
if os.path.exists(dataDir) != 1:
    os.mkdir(dataDir)       

#### FEATURE EXTRACTION ####

for fileName,parts in dataDictionary.items():
    #print(fileName)
    for part in parts:
        computeHPCP_FRAMEBASED(part,params)
        computeHPCP_GLOBAL(part) 
        
#Saving all results in a pickle file
pickleProtocol=1#choosen for backward compatibility
with open(os.path.join('data','HPCPFeaturesExtracted_ChordScaleDataset.pkl') , 'wb') as f:
    pickle.dump(dataDictionary, f, pickleProtocol)

#Let's clear data and re-read it
print('HPCP (Chroma) Features extracted and saved in a file')                

HPCP (Chroma) Features extracted and saved in a file


#### 2) NON-NEGATIVE LEAST SQUARED (NNLS) CHROMAS (Mauch, 2011)

#### coming soon